This script is used to generate the blue-to-red light gradeint direction vs. light/dark side, see Figure 4 and Figure 5 of Schallock & Hayes 2024
* x axis [-1,1] where values are distance-from-center/semi-minor-len. Positive numbers refer to dark sides per Iye. et al. 2019 and Negative number refer to 'bright side' (opposite of dark side).
* y axis [0,1] where 0 is the pixel of reddest value in difference image, 1 is the pixel of bluest value in difference image
* Only includes pixels within 45 degrees of minor axis
* Excludes trendline whose fitting likely failed (has min(y) <= -0.5 or max(y) >= 1.5), as well as cases where gofher label and spin parity labels are orthogonal.

In [82]:
import os

import sys
sys.path.insert(0, '../gofher')

import numpy as np
import matplotlib.pyplot as plt
import math
from collections import defaultdict

from gofher import run_gofher
from matrix import create_dist_matrix, create_centered_mesh_grid, create_minor_axis_angle_matrix
from spin_parity import read_spin_parity_galaxies_label_from_csv, standardize_galaxy_name, score_label


In [83]:
survery_to_use = "panstarrs"

BANDS_IN_ORDER = ['g','r','i','z','y'] #Important: Must stay in order of BLUEST to REDDEST Waveband (Editting this will cause gofher to no longer correctly evaluate redder side of galaxy)
REF_BANDS_IN_ORDER = ['i','z','y','r','g'] #The prefernce each waveband being choosen as refernce band from highest priority to lowest priority

In [84]:
#for formatting mnras paper: source: https://walmsley.dev/posts/typesetting-mnras-figures

SMALL_SIZE = 9
MEDIUM_SIZE = 9
BIGGER_SIZE = 9

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)
#plt.rc('font', family='Nimbus Roman No9 L') #uncomment this to use same font as MNRAS - requires font Nimbus Roman No9 L (can be downloaded from https://www.fontsquirrel.com/fonts/nimbus-roman-no9-l)

In [85]:
path_to_catalog_data = "..\\..\\spin-parity-catalog-data"
output_path_for_gradient_trend = "..\\..\\..\\gradient_vs_dark_side.png"

In [86]:
def get_galaxies(figure_to_run_on):
    return os.listdir(os.path.join(path_to_catalog_data,survery_to_use,figure_to_run_on))

In [87]:
def get_dark_side_csv_path(folder_name):
    csv_path = "C:\\Users\\school\\Desktop\\github\\spin-parity-catalog-data\\catalog"
    return os.path.join(csv_path,"{}.csv".format(folder_name))

def get_color_image_path(name,folder_name):
    return os.path.join(path_to_catalog_data,folder_name,name,"{}_color.jfif".format(name))

In [88]:
def seperate_out_for_gradient(the_gal, paper_label, poly_degree, max_ang_from_minor,linspace=100):
    x = the_gal.gofher_params.x; y = the_gal.gofher_params.x
    theta = the_gal.gofher_params.theta
    shape = the_gal[the_gal.ref_band].data.shape

    xv,yv = create_centered_mesh_grid(x,y,shape)
    dist = create_dist_matrix(xv,yv)
    ang = np.abs(create_minor_axis_angle_matrix(x,y,theta,shape))
    #ang[ang > max_ang_from_minor] = 0.0

    el_mask = the_gal.create_ellipse()
    pos_mask, neg_mask = the_gal.create_bisection()

    to_include = np.logical_and(el_mask,ang < max_ang_from_minor)
    b = the_gal.gofher_params.b

    
    to_return = dict()

    for band_pair_key in the_gal.band_pairs:
        band_pair = the_gal.get_band_pair(band_pair_key)
        diff_image = band_pair.diff_image
        diff_image[np.logical_not(to_include)] = -np.Inf

        ones = np.ones(shape)
        score = score_label(band_pair.classification_label, paper_label)
        paper_classification = band_pair.classification*score*-1 #*x-1 for issue with flip

        if paper_classification == 1:
            ones[neg_mask] *= -1
        elif paper_classification == -1:
            ones[pos_mask] *= -1
        else:
            return None

        xs = ((ones*dist)/b)[np.logical_not(np.isinf(diff_image))]
        ys = diff_image[np.logical_not(np.isinf(diff_image))]
        
        the_min = np.min(diff_image[np.logical_not(np.isinf(diff_image))])
        the_max = np.max(diff_image[np.logical_not(np.isinf(diff_image))])
        ys_normed = (ys-the_min)/(the_max-the_min)

        to_return[band_pair_key] = np.polynomial.Polynomial.fit(xs, ys_normed, poly_degree).linspace(linspace)

    return to_return

def plot_all_trends(all_trends, output_path="", poly_degree=10, linspace=100):
    keys = [['g-r', 'g-i'],
            ['g-z', 'g-y'],
            ['r-i', 'r-z'],
            ['r-y','i-z'],
            ['i-y','z-y']]
    first_fig = ['a','b','c','d','e','f','g','h','i','j']
    count = 0
    fig, axd = plt.subplot_mosaic(keys,figsize=(10/3, 7),
                                  constrained_layout=True,num=1, clear=True) #num=1, clear=True #https://stackoverflow.com/a/65910539/13544635
    fig.patch.set_facecolor('white')
    for band_pair in all_trends:
        xs = []
        ys = []
        for each in all_trends[band_pair]:
            x = each[0]
            y = each[1]
            mask = np.logical_and(x>-1.0,x<1.0)
            xs.extend(list(x[mask]))
            ys.extend(list(y[mask]))
            axd[band_pair].plot(x,y, alpha=0.25,linewidth=1.0,c='blue')

        aggregate_trend = np.polynomial.Polynomial.fit(xs, ys, poly_degree).linspace(linspace)
        axd[band_pair].plot(*aggregate_trend,linewidth=1.0,c='red', ls='--')

        axd[band_pair].axvline(0.0,c='black')

        
        i = np.argmin(aggregate_trend[1])
        min_x = aggregate_trend[0][i]
        axd[band_pair].axvline(min_x,c='red',linewidth=1.0)
        
        the_title = "({}) {}:".format(first_fig[count],band_pair)
        axd[band_pair].set_title(the_title)
        axd[band_pair].set_ylabel("||Difference||")
        axd[band_pair].set_xlabel("||Distance||")
        
        axd[band_pair].set_xlim([-1, 1])
        axd[band_pair].set_ylim([0, 1])
        count += 1
    if output_path != "":
        fig.savefig(output_path, dpi = 300, bbox_inches='tight')
        fig.clear()
        plt.close(fig)
    else:
        plt.show()


In [89]:
def run_on_folder_name(folder_name):
    paper_labels = read_spin_parity_galaxies_label_from_csv(get_dark_side_csv_path(folder_name))

    all_trends = defaultdict(list)

    i=1
    for name in get_galaxies(folder_name):
        print(i,name)

        if standardize_galaxy_name(name) not in paper_labels:
            print("skippimg",name)
            continue

        paper_label = paper_labels[standardize_galaxy_name(name)]

        get_fits_path = lambda name,band: os.path.join(path_to_catalog_data,survery_to_use,folder_name,name,"{}_{}.fits".format(name,band))

        try:
            the_gal = run_gofher(name,get_fits_path,BANDS_IN_ORDER,REF_BANDS_IN_ORDER, paper_label)
            gal_trend_lines = seperate_out_for_gradient(the_gal,paper_label,10,math.pi/4)
            
            if isinstance(gal_trend_lines,type(None)):
                print("skipping")
                continue
            to_include = True
            for each_band_pair in gal_trend_lines:
                x = gal_trend_lines[each_band_pair][0]
                y = gal_trend_lines[each_band_pair][1]
                if np.max(x) > 2.0 or np.min(x) < -2.0 or np.max(y) > 1.5 or np.min(y) < -0.5:
                    to_include = False
                    break

            if not to_include:
                print("skipping")
                continue

            for each_band_pair in gal_trend_lines:
                all_trends[each_band_pair].append(gal_trend_lines[each_band_pair])

        except Exception as e:
            print("Exception on gal",name,e)
        i += 1
        #if i > 10: break #you can uncomment this if you are debugging something and just want to run on a single galaxy
    return all_trends


In [90]:
observed_folders = ["figure8","figure10","figure11"] #use this for galaxies in which dark side was directly observed (see Iye et al. 2019)
inferred_folders = ["figure9"] #use this for galaxies in which dark side was infered (see Iye et al. 2019)

In [91]:
save_figure = False

In [92]:
if not os.path.exists(path_to_catalog_data):
    raise ValueError("The path to the catalog is not found {} - make sure you update path_to_catalog_data".format(path_to_catalog_data))

all_trends = defaultdict(list)

#for folder_name in observed_folders:
for folder_name in inferred_folders:
    print(folder_name)
    returned = run_on_folder_name(folder_name)
    for each_key in returned:
        all_trends[each_key].extend(returned[each_key])

output_pa = output_path_for_gradient_trend if save_figure else ""
plot_all_trends(all_trends,output_pa,10,100)


figure9


In [93]:
k = list(all_trends.keys())[0]
print(len(all_trends[k]))